# Text scrape Leeds City Council Election results into csv 

Data will be published on Leeds data mill but if you need it sooner than two weeks you can use the following code!

In [1]:
# import library for querying website
import pandas as pd
import os
import urllib
import requests
import html5lib
from bs4 import BeautifulSoup
from io import StringIO
from selenium import webdriver

In [2]:
# get list of links for scrapping

main_page = 'https://www.leeds.gov.uk/your-council/elections/leeds-city-council-election-results'

In [3]:
# set up selenium to use chrome in headless state
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
wd = webdriver.Chrome('tools/chromedriver', options=options)

wd.get(main_page)

html_page = wd.page_source

In [14]:
# opens page from selenium in beautiful soup
import re 

soup = BeautifulSoup(html_page, 'html.parser')

# finds specific section of page with links to each ward
accordion_layer = soup.find(id='acc_921')

In [16]:
accordion_layer.find_all('button', id=re.compile("acc_921_panel"))

[<button aria-controls="acc_921_panel_1" aria-expanded="false" class="accordion__trigger" data-current="false" id="acc_921_panel_1_trigger" type="button">  Adel &amp; Wharfedale   </button>,
 <button aria-controls="acc_921_panel_2" aria-expanded="false" class="accordion__trigger" data-current="false" id="acc_921_panel_2_trigger" type="button">Alwoodley</button>,
 <button aria-controls="acc_921_panel_3" aria-expanded="false" class="accordion__trigger" data-current="false" id="acc_921_panel_3_trigger" type="button">  Ardsley &amp; Robin Hood   </button>,
 <button aria-controls="acc_921_panel_4" aria-expanded="false" class="accordion__trigger" data-current="false" id="acc_921_panel_4_trigger" type="button">  Armley   </button>,
 <button aria-controls="acc_921_panel_5" aria-expanded="false" class="accordion__trigger" data-current="false" id="acc_921_panel_5_trigger" type="button">  Beeston &amp; Holbeck   </button>,
 <button aria-controls="acc_921_panel_6" aria-expanded="false" class="acco

In [7]:
# example extracting election results from a page
    
df_y = []

for link in final_list_links:
    
    # open webpage and get beautiful soup via selenium
    
    wd.get(link)

    html_page = wd.page_source
    
    # get page into bs
    soup = BeautifulSoup(html_page, 'lxml')

    # extract data table
    
    df = pd.read_html(html_page,header=0)[1]
    
    df['Share'] = round(((df['Votes'] / df['Votes'].sum())*100),1)
    
    # extract top row data on ward, turnout and electorate size
    top_row = soup.find(id="WebPartctl00_ctl46_g_5037fcd1_1720_4395_8f61_756022543ee2")
    
    top_row = top_row.text.strip()
    top_row = top_row[46:125]
    top_row = top_row.split(" ")
    
    # get turnout
    for i in top_row:
        if i == "%":
            break
        else:
            z = ''
            z += i
        
        df["Turnout"] = z
        j = ""
        # get total ward
        
    for i in top_row:
        if i == "Electorate":
            break
        else:
            j += " "+i
            df['Ward'] = j

        # get total electorate
    for i in top_row:
        if i == "Turnout":  
            break
        else:
            b = ""
            b += i
    df['Electorate'] = b
                
    # append dataframe to list of dataframes
    df_y.append(df)
    
df_y = pd.concat(df_y,axis=0, ignore_index=True)

In [8]:
df_y.to_csv("./Leeds_LE2019_results.csv")